# Databricks + Skyflow Integration with Unity Catalog

This notebook demonstrates integrating Skyflow tokenization and detokenization into Databricks using **Unity Catalog Batch Python UDFs**.

## Key Features

- ✅ **Batched execution** - High-throughput batching to Lambda, then Lambda batches to Skyflow
- ✅ **Persistent functions** - Functions stored in Unity Catalog, available across all clusters
- ✅ **Governed and shareable** - Fine-grained access control for tokenization and detokenization
- ✅ **Persistent views** - Create views that automatically tokenize/detokenize data
- ✅ **Production ready** - Perfect for ETL pipelines, BI tools, and team collaboration

## Architecture

```
Databricks → Lambda (batched) → Skyflow (batched)
```

- **Credentials**: Managed in Lambda (not in notebooks)
- **Batching to Lambda**: Configurable (default 500 rows per call for high throughput)
- **Lambda to Skyflow**: Automatic batching at 25 rows per Skyflow API call
- **Parallelization**: Spark automatically distributes across partitions

## Prerequisites

1. **Unity Catalog enabled** - Modern Databricks runtime (DBR 13.3+) or SQL warehouse
2. **Lambda function deployed** - See main README for deployment instructions
3. **Skyflow credentials** - Cluster ID, Vault ID, Table name

---

# Quick Start

1. **Configure cell 1** with your Lambda URL and Skyflow credentials
2. **Run cells 2-3** to create the persistent UDFs in Unity Catalog
3. **Run cells 4+** for usage examples and testing

The functions persist across cluster restarts and are available to all authorized users!

In [ ]:
# ============================================================================
# Step 1: Configuration - UPDATE THESE VALUES
# ============================================================================

# Unity Catalog location
CATALOG = "your_catalog_name"
SCHEMA = "your_schema_name"

# Lambda API configuration
LAMBDA_URL = "https://YOUR_API_ID.execute-api.YOUR_REGION.amazonaws.com/processDatabricks"

# Skyflow configuration
CLUSTER_ID = "YOUR_CLUSTER_ID"
VAULT_ID = "YOUR_VAULT_ID"
TABLE = "TABLE_NAME"

# Performance tuning
BATCH_SIZE = 500  # Rows per Lambda API call (Lambda then batches at 25 rows per Skyflow call)
REQUEST_TIMEOUT = 30  # HTTP timeout in seconds

print("=" * 60)
print("Configuration Summary")
print("=" * 60)
print(f"Catalog:     {CATALOG}")
print(f"Schema:      {SCHEMA}")
print(f"Lambda URL:  {LAMBDA_URL}")
print(f"Cluster ID:  {CLUSTER_ID}")
print(f"Vault ID:    {VAULT_ID}")
print(f"Table:       {TABLE}")
print(f"Batch Size:  {BATCH_SIZE} (to Lambda)")
print(f"Timeout:     {REQUEST_TIMEOUT}s")
print("=" * 60)
print("\n✓ Configuration loaded")
print("\nNote: Lambda internally batches at 25 rows per Skyflow API call")
print("\nNext: Run cells 2-3 to create batch Python UDFs")

In [ ]:
# ============================================================================
# Step 2: Create Tokenization Batch Python UDF
# ============================================================================
#
# This creates a persistent Unity Catalog function using PARAMETER STYLE PANDAS
# for efficient batched tokenization.
#
# IMPORTANT: Due to UC PARAMETER STYLE PANDAS limitations, you cannot pass literal
# strings directly. Use the derived column pattern shown below.
#

spark.sql(f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.skyflow_tokenize_column(
    column_value STRING,
    column_name STRING
)
RETURNS STRING
LANGUAGE PYTHON
PARAMETER STYLE PANDAS
HANDLER 'tokenize_handler'
AS $$
import pandas as pd
import requests
from typing import Iterator, Tuple

# Configuration embedded at function creation time
LAMBDA_URL = "{LAMBDA_URL}"
CLUSTER_ID = "{CLUSTER_ID}"
VAULT_ID = "{VAULT_ID}"
TABLE = "{TABLE}"
BATCH_SIZE = {BATCH_SIZE}
REQUEST_TIMEOUT = {REQUEST_TIMEOUT}

def tokenize_handler(batch_iter: Iterator[Tuple[pd.Series, pd.Series]]) -> Iterator[pd.Series]:
    '''
    Batch tokenization handler using Skyflow Lambda API.
    
    Args:
        batch_iter: Iterator yielding tuples of (values_series, column_names_series)
    
    Yields:
        Series of tokens for each batch
    '''
    for values, column_names in batch_iter:
        if column_names.empty:
            # No rows in this batch
            yield column_names
            continue
        
        # Extract column name (constant across batch)
        col_name = column_names.iloc[0]
        
        # Build records list
        records = []
        index_map = []  # Track indices of non-null values
        for idx, v in enumerate(values):
            if v is not None and pd.notna(v):
                records.append({{col_name: v}})
                index_map.append(idx)
        
        # If all values are null, return original
        if not records:
            yield values
            continue
        
        # Initialize results
        tokenized = [None] * len(values)
        
        # Process in sub-batches
        start = 0
        while start < len(records):
            end = min(start + BATCH_SIZE, len(records))
            batch_records = records[start:end]
            batch_indices = index_map[start:end]
            
            # Call Lambda API
            resp = requests.post(
                LAMBDA_URL,
                json={{"records": batch_records}},
                headers={{
                    "Content-Type": "application/json",
                    "X-Skyflow-Operation": "tokenize",
                    "X-Skyflow-Cluster-ID": CLUSTER_ID,
                    "X-Skyflow-Vault-ID": VAULT_ID,
                    "X-Skyflow-Table": TABLE
                }},
                timeout=REQUEST_TIMEOUT
            )
            resp.raise_for_status()
            
            # Parse response
            data = resp.json().get("data", [])
            
            # Map tokens back to original indices
            for local_i, rec in enumerate(data):
                global_idx = batch_indices[local_i]
                tokenized[global_idx] = rec.get(col_name)
            
            start = end
        
        # Yield result as Series
        yield pd.Series(tokenized)
$$
""")

print("✓ Created batch Python UDF: skyflow_tokenize_column(column_value, column_name)")
print(f"  Location: {CATALOG}.{SCHEMA}.skyflow_tokenize_column")
print(f"  Type: PARAMETER STYLE PANDAS (batched execution)")
print(f"  Batch Size: {BATCH_SIZE} rows per Lambda call")
print(f"  Lambda URL: {LAMBDA_URL}")
print()
print("=" * 70)
print("USAGE PATTERN - Derived Column Required")
print("=" * 70)
print()
print("⚠️  IMPORTANT: You cannot pass literal strings directly to UC Batch Python UDFs.")
print("   Use the 'derived column' pattern to work around this limitation:")
print()
print("✗ DOESN'T WORK:")
print("  SELECT skyflow_tokenize_column(email, 'email') FROM users")
print()
print("✓ CORRECT PATTERN:")
print("  WITH prepared_data AS (")
print("    SELECT email, 'email' AS email_col")
print("    FROM users")
print("  )")
print("  SELECT skyflow_tokenize_column(email, email_col)")
print("  FROM prepared_data")
print()
print("Multiple columns in one query:")
print("  WITH prepared_data AS (")
print("    SELECT")
print("      user_id,")
print("      email, 'email' AS email_col,")
print("      phone, 'phone' AS phone_col")
print("    FROM users")
print("  )")
print("  SELECT")
print("    user_id,")
print("    skyflow_tokenize_column(email, email_col) as email_token,")
print("    skyflow_tokenize_column(phone, phone_col) as phone_token")
print("  FROM prepared_data")

In [ ]:
# ============================================================================
# Step 3: Create Detokenization Batch Python UDF
# ============================================================================
#
# This creates a persistent Unity Catalog function using PARAMETER STYLE PANDAS
# for efficient batched detokenization.
#

spark.sql(f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.skyflow_detokenize(
    token STRING
)
RETURNS STRING
LANGUAGE PYTHON
PARAMETER STYLE PANDAS
HANDLER 'detokenize_handler'
AS $$
import pandas as pd
import requests
from typing import Iterator

# Configuration embedded at function creation time
LAMBDA_URL = "{LAMBDA_URL}"
CLUSTER_ID = "{CLUSTER_ID}"
VAULT_ID = "{VAULT_ID}"
BATCH_SIZE = {BATCH_SIZE}
REQUEST_TIMEOUT = {REQUEST_TIMEOUT}

def detokenize_handler(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    '''
    Batch detokenization handler using Skyflow Lambda API.
    
    Args:
        batch_iter: Iterator yielding pandas Series (one per batch)
    
    Yields:
        Series of detokenized values for each batch
    
    Note: Single-argument UDF receives Iterator[pd.Series], not Iterator[Tuple].
    '''
    for tokens in batch_iter:
        # tokens is a pandas Series for this batch
        if tokens.empty:
            yield tokens
            continue
        
        # Build mask for non-null tokens
        mask = tokens.notna()
        token_list = tokens[mask].tolist()
        
        if not token_list:
            # All nulls, return original
            yield tokens
            continue
        
        # Initialize output as copy of input
        output = tokens.copy()
        
        # Process in sub-batches
        start = 0
        while start < len(token_list):
            end = min(start + BATCH_SIZE, len(token_list))
            batch_tokens = token_list[start:end]
            
            # Call Lambda API
            resp = requests.post(
                LAMBDA_URL,
                json={{"tokens": batch_tokens}},
                headers={{
                    "Content-Type": "application/json",
                    "X-Skyflow-Operation": "detokenize",
                    "X-Skyflow-Cluster-ID": CLUSTER_ID,
                    "X-Skyflow-Vault-ID": VAULT_ID
                }},
                timeout=REQUEST_TIMEOUT
            )
            resp.raise_for_status()
            
            # Parse response and create token->value mapping
            data = resp.json().get("data", [])
            token_to_value = {{r["token"]: r["value"] for r in data}}
            
            # Write back into output Series only where not null
            sub_mask_idx = mask[mask].index[start:end]
            for idx, tok in zip(sub_mask_idx, batch_tokens):
                output.at[idx] = token_to_value.get(tok)
            
            start = end
        
        # Yield result as Series
        yield output
$$
""")

print("✓ Created batch Python UDF: skyflow_detokenize(token)")
print(f"  Location: {CATALOG}.{SCHEMA}.skyflow_detokenize")
print(f"  Type: PARAMETER STYLE PANDAS (batched execution)")
print(f"  Batch Size: {BATCH_SIZE} rows per Lambda call")
print(f"  Lambda URL: {LAMBDA_URL}")
print("\nThis function is now:")
print("  - Persistent in Unity Catalog")
print("  - Shareable across workspaces/users")
print("  - Usable in SQL queries and persistent views")
print("  - Batched for optimal performance")

## Setup Complete!

Two Unity Catalog Batch Python UDFs have been created:

1. **skyflow_tokenize_column(column_value, column_name)** - Batched tokenization
2. **skyflow_detokenize(token)** - Batched detokenization

These functions are now:
- ✅ Persistent in Unity Catalog
- ✅ Governed and shareable with proper access control
- ✅ Callable from SQL queries
- ✅ Usable in persistent views
- ✅ Batched for optimal throughput (configurable batch size to Lambda)

**Key Technology:** These functions use `PARAMETER STYLE PANDAS` which enables batched, vectorized execution while maintaining persistence and governance in Unity Catalog.

---

# Usage Examples

The cells below demonstrate how to use the batch Python UDFs.

## Generate Test Data

Create sample data for testing the functions:

In [ ]:
from pyspark.sql.functions import expr, current_timestamp

# Configure number of test rows
NUM_ROWS = 100

# Create test data
test_df = spark.range(NUM_ROWS).select(
    (expr("id + 1").alias("user_id")),
    expr("concat('user_', id)").alias("username"),
    expr("concat('user', id, '@example.com')").alias("email"),
    expr("concat('+1-555-', LPAD(id % 1000, 3, '0'), '-', LPAD((id * 7) % 10000, 4, '0'))").alias("phone"),
    current_timestamp().alias("created_at")
)

# Save as table
test_df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.raw_users")

print(f"✓ Created {CATALOG}.{SCHEMA}.raw_users table with {NUM_ROWS} rows")
display(spark.table(f"{CATALOG}.{SCHEMA}.raw_users").limit(10))

## Tokenize Data with Batch Python UDF

Use the batch Python UDF to tokenize email addresses efficiently.

**Note:** Due to Unity Catalog `PARAMETER STYLE PANDAS` limitations, you must use the **derived column pattern** when passing column names. The literal string `'email'` must be converted to a column expression using a subquery.

### Pattern Explanation

```sql
-- Convert literal 'email' to a column
SELECT
    skyflow_tokenize_column(email, column_name) as email_token
FROM (
    SELECT email, 'email' AS column_name
    FROM users
) t
```

The subquery creates a `column_name` column filled with `'email'`, which UC properly converts to a pandas Series in the UDF.

In [ ]:
# Tokenize using the batch Python UDF with derived column pattern
# Using CTE to ensure column_name is a proper column expression
result = spark.sql(f"""
    WITH prepared_data AS (
        SELECT
            user_id,
            username,
            email,
            phone,
            created_at,
            'email' AS column_name
        FROM {CATALOG}.{SCHEMA}.raw_users
    )
    SELECT 
        user_id,
        username,
        email,
        {CATALOG}.{SCHEMA}.skyflow_tokenize_column(email, column_name) as email_token,
        phone,
        created_at
    FROM prepared_data
""")

# Save tokenized data
result.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.tokenized_users")

print(f"✓ Created {CATALOG}.{SCHEMA}.tokenized_users table with tokenized emails")
print(f"  Tokenization was batched at {BATCH_SIZE} rows per Lambda API call")
display(spark.table(f"{CATALOG}.{SCHEMA}.tokenized_users").limit(10))

## Create Persistent Detokenized View

Create a **persistent view** (not possible with temporary UDFs!) that automatically detokenizes data:

In [ ]:
# Create a PERSISTENT view that detokenizes email tokens on-the-fly
spark.sql(f"""
    CREATE OR REPLACE VIEW {CATALOG}.{SCHEMA}.users_detokenized AS
    SELECT 
        user_id,
        username,
        {CATALOG}.{SCHEMA}.skyflow_detokenize(email_token) as email,
        phone,
        created_at
    FROM {CATALOG}.{SCHEMA}.tokenized_users
""")

print(f"✓ Created PERSISTENT view: {CATALOG}.{SCHEMA}.users_detokenized")
print("\n🎉 Key Achievement: This is a PERSISTENT view using batched detokenization!")
print("   - Not possible with temporary Pandas UDFs")
print("   - Much more efficient than scalar UC UDFs")
print(f"   - Batches at {BATCH_SIZE} rows per API call")
print("   - Accessible to all users with permissions")
print("\nQuerying the view:")
display(spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.users_detokenized LIMIT 10"))

## Verify Roundtrip Accuracy

Compare original values with tokenized and detokenized values:

In [ ]:
# Compare original vs detokenized
verification_df = spark.sql(f"""
    SELECT
        t.user_id,
        r.email as original_email,
        t.email_token,
        d.email as detokenized_email,
        CASE
            WHEN r.email = d.email THEN 'MATCH'
            ELSE 'MISMATCH'
        END as verification
    FROM {CATALOG}.{SCHEMA}.tokenized_users t
    JOIN {CATALOG}.{SCHEMA}.raw_users r ON t.user_id = r.user_id
    JOIN {CATALOG}.{SCHEMA}.users_detokenized d ON t.user_id = d.user_id
    LIMIT 10
""")

display(verification_df)

# Check for any mismatches
mismatches = verification_df.filter("verification = 'MISMATCH'").count()
if mismatches == 0:
    print("\n✓ All records match! Tokenization → Detokenization working correctly.")
    print("\n🎉 Batch Python UDFs are working perfectly with batched Lambda calls!")
else:
    print(f"\n✗ Found {mismatches} mismatches - investigate!")

## Grant Permissions (Optional)

Control access to the batch Python UDFs:

In [ ]:
# Example: Grant function execution permissions
# Uncomment and adjust as needed

# spark.sql(f"GRANT EXECUTE ON FUNCTION {CATALOG}.{SCHEMA}.skyflow_tokenize_column TO `data_engineers`")
# spark.sql(f"GRANT EXECUTE ON FUNCTION {CATALOG}.{SCHEMA}.skyflow_detokenize TO `data_engineers`")
# spark.sql(f"GRANT SELECT ON VIEW {CATALOG}.{SCHEMA}.users_detokenized TO `analysts`")

print("💡 Tip: Use Unity Catalog's access control to govern who can:")
print("   - Execute tokenization (potentially create PII tokens)")
print("   - Execute detokenization (access plaintext PII)")
print("   - Query detokenized views (read PII)")

## Cleanup (Optional)

To remove all test resources:

In [ ]:
# Uncomment to drop test tables, views, and functions
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.raw_users")
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.tokenized_users")
# spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{SCHEMA}.users_detokenized")
# spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{SCHEMA}.skyflow_detokenize")
# spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{SCHEMA}.skyflow_tokenize_column")
# print("✓ Cleanup complete")

## Summary

This notebook demonstrated **Unity Catalog Batch Python UDFs** for Skyflow integration in Databricks.

### Key Achievements

✅ **Batched Execution**
- Configurable batch size to Lambda (default 500 rows per call for high throughput)
- Lambda internally batches at 25 rows per Skyflow API call
- Dramatically reduces API costs compared to row-by-row processing

✅ **Persistent & Governed**
- Functions stored in Unity Catalog
- Shareable across workspaces and users
- Fine-grained access control

✅ **Production Ready**
- Usable in persistent views
- Callable from SQL queries
- Survives cluster restarts
- Perfect for BI tools and team collaboration

### The Magic: PARAMETER STYLE PANDAS

```sql
CREATE FUNCTION ... 
LANGUAGE PYTHON
PARAMETER STYLE PANDAS  -- This enables batched execution!
HANDLER 'handler_function'
```

This directive tells Databricks to:
1. Group rows into batches (pandas Series)
2. Call the handler function with batches (not individual rows)
3. Process results as batches

### Handler Signature Patterns

**Single-argument UDF** (e.g., detokenization):
```python
def handler(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for values in batch_iter:
        # Process batch
        yield pd.Series(results)
```

**Two-argument UDF** (e.g., tokenization):
```python
def handler(batch_iter: Iterator[Tuple[pd.Series, pd.Series]]) -> Iterator[pd.Series]:
    for arg1, arg2 in batch_iter:
        # Process batch
        yield pd.Series(results)
```

### Performance Notes

- **Batch Size to Lambda:** Default 500 rows per call (configurable via BATCH_SIZE parameter)
- **Lambda to Skyflow:** Automatic internal batching at 25 rows per Skyflow API call
- **Timeout:** 30 seconds (must be ≤ Lambda timeout)
- **Parallelization:** Spark automatically parallelizes across partitions
- **Scalability:** High batch size to Lambda allows Lambda to scale out and churn through data quickly

For 100K rows with BATCH_SIZE=500:
- ~200 Lambda calls
- Lambda then makes ~4,000 Skyflow API calls (at 25 rows each)
- Compare to row-by-row scalar UDFs: 100,000 Lambda calls (500x more expensive!)

### Derived Column Pattern

Due to Unity Catalog limitations, you cannot pass literal strings directly to `PARAMETER STYLE PANDAS` functions. Use the derived column pattern:

```sql
-- Convert literal to column
WITH prepared AS (
  SELECT email, 'email' AS email_col
  FROM users
)
SELECT skyflow_tokenize_column(email, email_col)
FROM prepared
```

### Next Steps

1. **Monitor Performance:** Check Lambda CloudWatch logs for batch sizes and timing
2. **Tune Batch Size:** Adjust BATCH_SIZE based on your data size, network latency, and Lambda timeout
3. **Set Up Governance:** Grant appropriate permissions to users/groups
4. **Create Views:** Build persistent views for your BI tools and analysts